In [1]:
# making changes to tempdata

In [2]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")

In [3]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *

### etl (new readfilerun and gen)

In [4]:
def readfilerun(run_):#, output_dir):
    outdir = os.path.join(os.getcwd() , "data", "tempdata")
    names = listdir('data') # all filenames in data
    daneruns = [x for x in names if not 'losslog' in x]
    daneruns.remove('tempdata')
    
    # daneruns = ['data/20220116T055105_20-100-true-20-100-iperf.csv', 'data/20220116T055942_20-250-true-20-250-iperf.csv']
    losslogs = [x for x in names if 'losslog' in x]
    #print(losslogs)
    for count, run in enumerate(daneruns):
        print(count)
        # print(run) #for debug
        curr_losslog = losslogs[count]
        print("run: ", run)
        run_labels = run.split('_')[1].split('.')[0].split('-')
        print("run_labels:", run_labels)
        temp_label_str = '-'.join(run_labels) 
        #print(temp_label_str)
        losslog = f'data/{curr_losslog}' #losslog filename str
        #print(f'data3/{run}')
        #print(losslog)
        run_df = pd.read_csv(f'data/{run}')
        losslog_df = pd.read_csv(losslog, header=None).rename(
            columns={0:'event', 1:'drop_unix', 2:'IP1', 3:'Port1', 4:'IP2', 5:'Port2', 6:'Proto'})
        losslog_df['Time'] = losslog_df['drop_unix'].astype(int)
        df = pd.merge(run_df, losslog_df, on=['Time', 'IP1', 'Port1', 'IP2', 'Port2', 'Proto'], how="left") # merge on fivetuple key
        # df.fillna(inplace=True, value=-1) #TODO plan implementation of dealing with null values
        # df['event'] =  # TODO change to 3 different profiles: no drop, drop, and switch event
        df = df[df['Proto'] == df['Proto'].mode()[0]] # selects relevant non ipv6 int(connection
        
        ## adding labels
        df['latency'] = int(run_labels[0])
        df['loss'] = int(run_labels[1])
        df['later_latency'] = int(run_labels[3])
        df['later_loss'] = int(run_labels[4])
        df['deterministic'] = bool(run_labels[2])
        
        #TODO future implementation of boolean flag for when the switch happens so we know when to use later lat/loss
        
        df.to_csv(f'{outdir}/labeled-from-{run_}_{temp_label_str}.csv') # save to temporary output directory: just merging takes a bit

In [5]:
def gen(cond , subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"
    tempdatafiles = listdir('data/tempdata')
    fnames = [ filename for filename in tempdatafiles if filename.endswith(".csv" ) ]
    
    data, datasubset, transformed  = [], [], []
    for j in fnames:
        loc = os.path.join(os.getcwd(), 'data', "tempdata", j)
        print(loc)
        df_cols = genfeat(pd.read_csv(loc))
        
        #data
        time_scaled = time(df_cols)
        data.append(time_scaled)
        
        #subset
        df_mid = time_scaled.iloc[60:60+subset]
        datasubset.append(df_mid)

        #transformed
        f_df = agg10(df_cols)
        transformed.append(f_df)
        
    # makes paths
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    
    list_to_csv(data, os.path.join(path2, unseen + "combined_all.csv"))
    print('combined_finished', sep=' ')
    list_to_csv(datasubset, os.path.join(path2, unseen + "combined_subset_6068.csv"))
    print('combined_all_finished', sep=' ')
    list_to_csv(transformed, os.path.join(path2, unseen +  "combined_transform.csv"))
    print('transformed_finished', sep=' ')
        
    return None
def list_to_csv(lst, filepth):
    '''takes list of pandas dataframes with similar column structure and outputs them to a single folder'''
    lst[0].to_csv(filepth, index=False)
    for i in range(1, len(lst)):
        lst[i].to_csv(filepth, index=False, header=False, mode='a')
    return


In [6]:
def genviz(cond, fname, subset):
    unseen = ""
    if cond == 'seen': 
        print("transforming seen data")
    if cond == 'unseen': 
        print("transforming un seen data")
        unseen = "unseen"

    data, datasubset, transformed  = [], [], []
    #loc = os.path.join(os.getcwd(),"data", "tempdata", fname)
    loc =  os.path.join(os.getcwd(),"tempdata2", fname)
    
    t = pd.read_csv(loc)
    df_cols = genfeat(t)
    print(fname)
    run_labels = fname.split('_')[1].split('.')[0].split('-')
    print(run_labels)

    #data
    time_scaled = time(df_cols)
    data.append(time_scaled)

    #subset
    df_mid = time_scaled.iloc[:subset]
    datasubset.append(df_mid)

    #transformed
    f_df = agg10(df_cols)
    transformed.append(f_df)
    
    path = os.path.join(os.getcwd() , "outputs", "gen_temp")
    path2 = os.path.join(os.getcwd() , "outputs")
    label = run_labels[0] + '-' + run_labels[1] +  '__' + run_labels[3] + '-' + run_labels[4]
    temp_data = pd.concat(data , ignore_index=True)#.reset_index(drop = True)
    temp_data.to_csv(os.path.join(path2, unseen + "s_all_" + label + ".csv"), index = False)
    
    temp_subset = pd.concat(datasubset , ignore_index=True)
    temp_subset.to_csv(os.path.join(path2, unseen +  "s_subset_" + label + ".csv"), index = False)
    
    temp_t = pd.concat(transformed, ignore_index=True)  
    temp_t.to_csv(os.path.join(path2, unseen +  "s_transform_" + label + ".csv"), index = False)

#### eda

In [7]:
def main_eda(cond, lst, filen1, filen2, filen3):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    
    fpath1 = os.path.join(os.getcwd() , "outputs", unseen + filen1)
    df_1 = pd.read_csv(fpath1)
    fpath2 = os.path.join(os.getcwd() , "outputs", unseen + filen2)
    df_2 = pd.read_csv(fpath2)
    fpath3 = os.path.join(os.getcwd() , "outputs", unseen + filen3)
    df_3 = pd.read_csv(fpath3)
    
    plottogether(cond, lst, df_1, filen1.strip(".csv")) # trends over subset
    plottogether(cond, lst, df_3, filen3.strip(".csv")) # trends over entire data
    plotloss(cond, df_2)

    plot_correlation_matrix(cond, df_2) # correlation matrix
    plotlongest(df_3, cond, 600, 1400)
    # below makes rest of visualizations
    plotbytes(df_3, 600, 1400, 200, 200)
    #plot_detailed_bytes()

In [8]:
# overlaying model predictions and plotting detailed bytes

#### train

In [9]:
def test_feat(cond, df, cols, p, df_u): 
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    # col is feauture comb
    # p is for loss or latency   1: loss  # 2 : latency
    X = df[cols]
    
    X2 = df_u[cols]

    if p == 1:  # flag found in test_mse
        y = df.loss
        y2 = df_u.loss
    if p == 2: 
        y = df.latency
        y2 = df_u.latency
        
    # randomly split into train and test sets, test set is 80% of data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    if unseen == 'unseen': 
        X_test = X2
        y_test = y2
    
    clf = DecisionTreeRegressor()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    acc1 = mean_squared_error(y_test, y_pred)
    
    clf2 = RandomForestRegressor(n_estimators=10)
    clf2 = clf2.fit(X_train,y_train)
    y_pred2 = clf2.predict(X_test)
    acc2= mean_squared_error(y_test, y_pred2)
    
    clf3 = ExtraTreesRegressor(n_estimators=10)
    clf3 = clf3.fit(X_train,y_train)
    y_pred3 = clf3.predict(X_test)
    acc3= mean_squared_error(y_test, y_pred3)
    
    pca = PCA() 
    X_transformed = pca.fit_transform(X_train) 
    cl = DecisionTreeRegressor() 
    cl.fit(X_transformed, y_train)
    newdata_transformed = pca.transform(X_test)
    y_pred4 = cl.predict(newdata_transformed)
    acc4 = mean_squared_error(y_test, y_pred4)
    
    clf_gbc = GradientBoostingRegressor(random_state=0)
    clf_gbc.fit(X_train, y_train)
    y_pred5 = clf_gbc.predict(X_test)
    acc5 = mean_squared_error(y_test, y_pred5) 
    return [acc1, acc2, acc3, acc4, acc5]

def train_feat()

def test_mse(cond, all_comb1, all_comb2):
    unseen = ''
    if cond =='unseen': 
        unseen = 'unseen'
    filedir_unseen = os.path.join(os.getcwd(), "outputs", unseen + "combined_transform.csv")
    df_unseen = pd.read_csv(filedir_unseen)
    filedir = os.path.join(os.getcwd(), "outputs", "combined_transform.csv")
    df = pd.read_csv(filedir)
    
    all_comb1 = pd.Series(all_comb1).apply(lambda x: list(x))
    all_comb2 = pd.Series(all_comb2).apply(lambda x: list(x))
    
    dt = []
    rf = []
    et = []
    pca = []
    gbc = []
    for i in all_comb1:
        acc_loss = test_feat(cond, df, i, 1, df_unseen)
        dt.append(acc_loss[0])  
        rf.append(acc_loss[1])  
        et.append(acc_loss[2])   
        pca.append(acc_loss[3])   
        gbc.append(acc_loss[4])
        
    # optimze by adding a flag called losslat to avoid making two dataframes of results
    dt2 = []
    rf2 = []
    et2 = []
    pca2 = []
    gbc2 = []
    for i in all_comb2:
        # 1 = loss
        # 2 = latency
        acc_latency = test_feat(cond, df, i, 2, df_unseen)
        dt2.append(acc_latency[0])
        rf2.append(acc_latency[1])
        et2.append(acc_latency[2]) 
        pca2.append(acc_latency[3])
        gbc2.append(acc_latency[4])
        
    dict1 = pd.DataFrame({'feat': all_comb1, 'dt': dt, 'rf': rf, 'et': et, 'pca': pca, 'gbc': gbc})
    dict2 = pd.DataFrame({'feat2': all_comb2, 'dt2': dt2, 'rf2': rf2, 'et2': et2, 'pca2': pca2, 'gbc2': gbc2})
    
    path = os.path.join(os.getcwd() , "outputs")
    dict1.to_csv(os.path.join(path, unseen + "feat_df1.csv"), index = False)
    dict2.to_csv(os.path.join(path, unseen + "feat_df2.csv"), index = False)


def best_performance(cond):
    unseen = ''
    if cond == 'unseen': 
        unseen = 'unseen'
    #print("finding best loss performance")
    filedir1 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df1.csv")
    df1 = pd.read_csv(filedir1)
    print( "\n")
    print("Loss Performance sorted from lowest to highest", "\n")
    print(df1.sort_values(by=['dt', 'rf', 'et', 'pca'], ascending = True)[:5], "\n")
    
    #print("finding best latency performance")
    filedir2 = os.path.join(os.getcwd(), "outputs", unseen + "feat_df2.csv")
    df2 = pd.read_csv(filedir2)
    print( "\n")
    print("Latency Performance sorted from lowest to highest", "\n")
    print(df2.sort_values(by=['dt2', 'rf2', 'et2', 'pca2'], ascending = True)[:5], "\n")


#### main

In [10]:
def main(targets):

    transform_config = json.load(open('config/transform.json'))
    columns = json.load(open('config/columns.json'))
    eda_config = json.load(open('config/eda.json'))
    all_config = json.load(open("config/all.json"))

    test_unseen = 'unseen'
    test_seen = 'seen'
    
    cond1 = True
    cond2 = False

    if 'data' in targets:
        """generating feat from unseen and seen data"""
#         readfilerun('data', 'tempdata') 
        gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)

    if 'eda' in targets:  
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
        main_eda(test_seen, [200, 200], **eda_config)
        print("EDA saved to outputs/eda/ folder")

    if 'train' in targets:
        "trains tests in this target"
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
                
        comb1 = getAllCombinations(1)
        comb2 = getAllCombinations(2)
        
        print("Testing on seen data: ")
        test_mse(test_seen, comb1, comb2)
        best_performance(test_seen)
                        
    if "inference" in targets: 
        print('tba')
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
            
    if "test" in targets: 
        """ runs all targets on sample data"""
        print('tba')
#         readfilerun('data', 'tempdata') 
#         gen(test_seen, **transform_config)
#         gen(test_unseen, **transform_config)
        
#         main_eda(test_seen, **eda_config)
#         print("EDA saved to outputs/eda/ folder")
        
#         comb1 = getAllCombinations(1)
#         comb2 = getAllCombinations(2)
        
#         print("Testing on seen data: ")
#         test_mse(test_seen, comb1, comb2)
#         best_performance(test_seen)
        
#         print("Testing on unseen data: ")
#         test_mse(test_unseen, comb1, comb2)
#         best_performance(test_unseen)
        
    if 'all' in targets: 
        print('tba')
        
# if __name__ == '__main__':

#     targets = sys.argv[1:]
#     main(targets)

In [11]:
targets = ['data', 'eda', 'train', 'inference', 'test', 'all']
# main(targets[0])

In [13]:
# main(targets[2])

MAKING pipeline: , tst on data with change in loss later

* https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
* https://stackoverflow.com/questions/58781601/parameter-tuning-using-gridsearchcv-for-gradientboosting-classifier-in-python
    
    comb[some index]
    
    dt() - untrained
    
    dt( comb) - train, or grid search on it
    
    et() - 
    
    plot them on top of eachother
    
    test(single dataset - aggregated over subset )# overserved or not
    
    plot the bytes, loss records (loss log), and expanded model predictions

In [27]:
path = os.path.join(os.getcwd() , "outputs")

comb1 = getAllCombinations(1)
comb2 = getAllCombinations(2)

fname = 'labeled-from-data_20-1000-true-20-1000.csv'
s_t = pd.read_csv('data\\tempdata2\\labeled-from-data_20-1000-true-20-1000.csv')

agged = agg10(genfeat(s_t))
# bob = pd.read_csv(os.path.join(os.getcwd() , "data/tempdata2", fname))
# bob['Seconds'] = bob['Time'] - min(bob['Time'])

# bob2 = pd.read_csv(os.path.join(os.getcwd() , 'outputs', newfname))

# X, y = s_t[comb1[4]], s_t['loss']

# # combined csv
# X_all = pd.read_csv(os.path.join(os.getcwd() , 'outputs', 'combined_transform.csv'))
# y_all = X_all['loss']

# # traintestsplit
# X_train, X_test, y_train, y_test = train_test_split(X_all[comb1[4]], y_all,
                                                #    random_state=0)

all combinations generated
all combinations generated


In [34]:
print(agged.shape)
agged.head()

(24, 17)


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,1327754.0,5680.776535,6.0,1303500.0,24254.0,869.0,451.7,1320.7,261.4,1007.641023,890.9,1327754.0,5.6,1327754.0,1042.514417,1000.0,20.0
1,1327754.0,5680.776535,6.0,1303500.0,24254.0,869.0,451.7,1320.7,261.4,1007.641023,890.9,1327754.0,5.6,1327754.0,1042.514417,1000.0,20.0
2,1327754.0,5680.776535,6.0,1303500.0,24254.0,869.0,451.7,1320.7,261.4,1007.641023,890.9,1327754.0,5.6,1327754.0,1042.514417,1000.0,20.0
3,1327754.0,5680.776535,6.0,1303500.0,24254.0,869.0,451.7,1320.7,261.4,1007.641023,890.9,1327754.0,5.6,1327754.0,1042.514417,1000.0,20.0
4,1327754.0,5680.776535,6.0,1303500.0,24254.0,869.0,451.7,1320.7,261.4,1007.641023,890.9,1327754.0,5.6,1327754.0,1042.514417,1000.0,20.0


In [35]:
print(s_t.shape)
s_t.head()

(387, 31)


,Unnamed: 0,Time,IP1,Port1,IP2,Port2,Proto,1->2Bytes,2->1Bytes,1->2Pkts,...,total_bytes,total_pkts,pkt sum,longest_seq,number_ms,max_bytes,total_pkt_sizes,pkt_ratio,time_spread,byte_ratio
0,0,1642313442,172.29.0.3,36174,172.26.0.2,5001,6,480000,9360,320,...,489360,500,489360,4,64,24208,489360,978.720000,192,978.720000
1,1,1642313443,172.29.0.3,36174,172.26.0.2,5001,6,2269500,44512,1513,...,2314012,2369,2314012,6,349,15260,2314012,976.788518,999,976.788518
2,2,1642313444,172.29.0.3,36174,172.26.0.2,5001,6,1642500,31936,1095,...,1674436,1666,1674436,5,308,16820,1674436,1005.063625,950,1005.063625
3,3,1642313444,172.29.0.3,36174,172.26.0.2,5001,6,1642500,31936,1095,...,1674436,1666,1674436,5,308,16820,1674436,1005.063625,950,1005.063625
4,4,1642313444,172.29.0.3,36174,172.26.0.2,5001,6,1642500,31936,1095,...,1674436,1666,1674436,5,308,16820,1674436,1005.063625,950,1005.063625


In [37]:
>>> from sklearn import svm, datasets
>>> from sklearn.model_selection import GridSearchCV
>>> iris = datasets.load_iris()
>>> parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
>>> svc = svm.SVC()
>>> clf = GridSearchCV(svc, parameters)
>>> clf.fit(iris.data, iris.target)
GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
>>> sorted(clf.cv_results_.keys())
['mean_fit_time', 'mean_score_time', 'mean_test_score',...
 'param_C', 'param_kernel', 'params',...
 'rank_test_score', 'split0_test_score',...
 'split2_test_score', ...
 'std_fit_time', 'std_score_time', 'std_test_score']

SyntaxError: invalid syntax (<ipython-input-37-5ccddd4d8a54>, line 12)

In [40]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

#creating Scoring parameter: 
scoring = {'mse': make_scorer(mean_squared_error)
          }
# feature paramter??

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

# set the tolerance to a large value to make the example faster
gradientbooster = GradientBoostingRegressor()
# depth and length?
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("gbt", gradientbooster)])

X_digits, y_digits = X_all[comb1[4]], y_all
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    "pca__n_components": [5, 15, 30, 45, 60],
    "gbt__max_depth": [3, 6, 9],
}
search = GridSearchCV(pipe, param_grid, n_jobs=2,scoring=scoring, refit= False)

NameError: name 'X_all' is not defined

In [ ]:
X_digits.head()

,total_bytes,max_bytes,longest_seq
0,2094730.0,168734.880609,268.7
1,2094730.0,168734.880609,268.7
2,2094730.0,168734.880609,268.7
3,2094730.0,168734.880609,268.7
4,2094730.0,168734.880609,268.7


In [ ]:
print(X_digits.info())
print(X_digits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bytes  117 non-null    float64
 1   max_bytes    117 non-null    float64
 2   longest_seq  117 non-null    float64
dtypes: float64(3)
memory usage: 2.9 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bytes  117 non-null    float64
 1   max_bytes    117 non-null    float64
 2   longest_seq  117 non-null    float64
dtypes: float64(3)
memory usage: 2.9 KB
None


In [ ]:
search.fit(X_digits, y_digits)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/opt/conda/lib/python3.9/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/opt/conda/lib/python3.

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('gbt', GradientBoostingRegressor())]),
             n_jobs=2,
             param_grid={'gbt__max_depth': [3, 6, 9],
                         'pca__n_components': [5, 15, 30, 45, 60]},
             refit=False, scoring={'mse': make_scorer(mean_squared_error)})

s/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/opt/conda/lib/python3.9/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/decomposition/_pca.py", line 383, in fit_transform
    U, S, Vt = self._fit(X)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/decomposition/_pca.py", line 430, in _fit
    return self._fit_full(X, n_components)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/decomposition/_pca.py", line 446, in _fit_full
    raise ValueError("n_components=%r must be between 0 and "
ValueError: n_components=15 must be between 0 and min(n_samples, n_features)=3 with svd_solver='full'

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/conda/

In [ ]:

print("Best parameter (CV score=%0.3f):" % search.best_score)
print(search.best_params)

# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    search.best_estimator.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/opt/conda/lib/python3.9/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/opt/conda/lib/python3.

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [ ]:
! conda update -n base conda

In [ ]:
! conda update scikit-learn

In [ ]:
search

In [ ]:
search.get_params

In [ ]:
search.

In [36]:
pipe = Pipeline([('scaler', StandardScaler()), ('svc', RandomForestRegressor())])
pipe.fit(X_train, y_train)
p = pipe.predict(X_test)
p_2 = pipe.predict(X)
mean_squared_error(y_test, p) #pipe.score(X_test, y_test)

NameError: name 'Pipeline' is not defined

#### new plot (overlay bytes and preds)

In [ ]:
df_small = pd.DataFrame(p_2).reset_index()
df_small['index'] = df_small['index'].apply(lambda x: list(range(0, 17)))
#print(df_small)
df_small = df_small.explode('index').reset_index().rename(columns={"index": "Seconds", 0: "loss"})
overlay = pd.DataFrame(df_small[['loss', 'Seconds']])
overlay_17 = overlay[17:]



In [ ]:
overlay_17.dtypes

In [ ]:
bobble = pd.merge( overlay_17, bob, on='Seconds')

In [ ]:
from matplotlib.lines import Line2D     
def plot_detailed_bytes(df, col='1->2Bytes', rollsec=10, labelcol ='loss_x'):
    rollcolor = '#6c2b6d'
    detailcolor = '#e98d6b'
    
    ax = plt.figure(figsize=(18,8))
    df[col].plot(title=f'{col}/s Rate', color=detailcolor)
    df[col].rolling(rollsec).mean().bfill().plot(color=rollcolor)
    plt.axvline(x=180, color='r')
    
    for i in df[df['event'] == 'drop'].index:
        
        plt.axvline(x=i, color='y', alpha=.45)
    custom_lines = [Line2D([0], [0], color=detailcolor, lw=2),
        Line2D([0], [0], color=rollcolor, lw=2),
        Line2D([0], [0], color='y', lw=2, alpha=0.45),
        Line2D([0], [0], color='r', lw=2)]
    plt.legend(custom_lines, 
               [f'{col} per Second', f'{col} per Second ({rollsec}s rolling avg)', 'Packet drop', '180s Mark'], 
               loc='upper right', framealpha=1);

In [ ]:
bobble.head()

In [ ]:
import matplotlib.lines
plot_detailed_bytes(bobble)

In [ ]:
#clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring=scoring,refit=False,cv=2, n_jobs=-1)